# qSDK VQE: Custom Ansatz Tutorial
The qSDK comes packaged with an implementation of both the UCCSD, and rUCC ansatze. In this tutorial, we'll explore how you can incorporate the built in VQESolver into your own workflow, by introducing a user-defined custom ansatz circuit. We'll base our work here on the *VQESolver* class, and take advantage of tools available through the qSDK and *agnostic_simulator* package. 

In [3]:
import numpy as np
from qsdk.electronic_structure_solvers.vqe_solver import VQESolver
from qsdk.toolboxes.ansatz_generator.ansatz import Ansatz
from agnostic_simulator import Circuit, Gate

In the VQESolver, we are expecting an instance of an abstract Ansatz class, which will be responsible for constructing the variational circuit we use to minimize the energy of our problem. For our example, I'm going to implement the so-called [Hardware Efficient Ansatz](https://arxiv.org/pdf/1704.05018.pdf) (HEA), developed by Kandala et al at IBM. In this ansatz, a circuit is constructed with repeated layers of a simple structure. Each layer consists of entangling gates (e.g. CNOT or CZ) which couple neighbouring qubits, followed by a series of Euler rotations carried out as single-qubit rotations $\mathrm{exp}\{i\theta_i^1 Z_i\}\mathrm{exp}\{i\theta_i^2 X_i\}\mathrm{exp}\{i\theta_i^3 Z_i\}$. We'll start by initializing our *Ansatz* class, and then fill in the functionality required to implement in *VQESolver*.

To construct our HEA ansatz, we're going to make use of the two helper functions defined here. The first will go through a register of qubits and add a layer of Euler-rotations as prescribed above. The second adds two columns of alternating CNOT gates, establishing long-range entanglement.

In [54]:
def EulerCircuit(n_qubits):
    """Construct a circuit applying an Euler Z-X-Z rotation to each qubit."""
    circuit = Circuit()
    for target in range(n_qubits):
        circuit.add_gate(Gate("RZ" , target, parameter=0.0, is_variational=True))
        circuit.add_gate(Gate("RX", target, parameter=0.0, is_variational=True))
        circuit.add_gate(Gate("RZ", target, parameter=0.0, is_variational=True))
    return circuit

def Entanglers(n_qubits):
    """Construct a circuit applying two columns of staggered CNOT gates to all qubits
     and their neighbours"""
    circuit = Circuit()
    for ii in range(n_qubits//2):
        circuit.add_gate(Gate("CNOT", control=2*ii, target=2*ii + 1))
    for ii in range(n_qubits//2 - 1):
        circuit.add_gate(Gate("CNOT", control=2*ii + 1, target=2*(ii+1)))
    return circuit

To build up our own Ansatz class, we'll require the following.

0. *__init__*: an initialization function to instantiate the class.
1. *set_var_params*: initialize the variational circuit parameters
2. *update_var_params*: update the parametric gates in the circuit
3. *build_circuit*: instantiate the variational circuit object
4. *prepare_reference_state*: get fixed circuit for initializing the reference, e.g. HF state.

Below, we're going to type out the entire class as we will use it. This is a lot of code in one place. So afterwards, we'll break it down into each of the relevant member methods.

In [62]:
class HEA(Ansatz):

    def __init__(self, n_qubits, n_electrons, n_layers, mapping='jw'):

        self.n_qubits = n_qubits
        self.n_electrons = n_electrons
        #number of layers of repeated entangler + Euler rotations
        self.n_layers = n_layers
        
        #specify fermion-to-qubit mapping (required for the initial reference state)
        self.mapping = mapping
        
        #Each layer has 3 variational parameters per qubit, and one non-variational entangler
        #There is an additional layer with no entangler.
        self.n_var_params = self.n_qubits * 3 * (self.n_layers + 1)

        self.var_params = None
        self.circuit = None

    def set_var_params(self, var_params=None):
        """Set initial variational parameter values"""
        if var_params is None:
            var_params = np.random.random(self.n_var_params)
        elif var_params.size != self.n_var_params:
            raise ValueError('Invalid number of parameters.')
        self.var_params = var_params
        return var_params

    def update_var_params(self, var_params):
        """Update variational parameters (done repeatedly during VQE)"""
        for param_index in range(self.n_var_params):
            self.circuit._variational_gates[param_index].parameter = var_params[param_index]
    
    def prepare_reference_state(self):
        """Prepare a circuit generating the HF reference state"""
        return get_reference_circuit(n_qubits=self.n_qubits,
                                     n_electrons=self.n_electrons,
                                     mapping=self.mapping)

    def build_circuit(self, var_params=None):
        """Construct the variational circuit to be used as our ansatz."""
        self.var_params = self.set_var_params(var_params)

        circuit = EulerCircuit(self.n_qubits)
        for ii in range(self.n_layers):
            circuit += Entanglers(self.n_qubits)
            circuit += EulerCircuit(self.n_qubits)

        reference_state_circuit = self.prepare_reference_state()
        
        if reference_state_circuit.size != 0:
            self.circuit = reference_state_circuit + circuit
        else:
            self.circuit = circuit
        return circuit

Now let's parse out what's happening in each of these member methods we just defined. Let's start with *set_var_params*. I'm going to do something very simple and just force this to be a random numpy array. 

In [56]:
def set_var_params(self, var_params=None):

    if var_params is None:
        var_params = np.random.random(self.n_var_params)
    elif var_params.size != self.n_var_params:
        raise ValueError('Invalid number of parameters.')
    self.var_params = var_params
    return var_params


Next, we'll implement *update_var_params*, where the circuit is updated with a new batch of variational parameters. All variational gates in the circuit are updated as required.

In [57]:
def update_var_params(self, var_params):

    for param_index in range(self.n_var_params):
        self.circuit._variational_gates[param_index].parameter = var_params[param_index]

Next, we'll use the methods from the *qubit_mappings* toolbox to construct a Hartree-Fock reference state.

In [39]:
from qsdk.toolboxes.qubit_mappings.statevector_mapping import get_reference_circuit

def prepare_reference_state(self):

    return get_reference_circuit(n_qubits=self.n_qubits,
                                         n_electrons=self.n_electrons,
                                         mapping=self.mapping)

Finally, we'll implement the *build_circuit* method. As compared to the three others here, this is really the only method in the present case that requires much thought. Evidently, everything above has followed pretty boilerplate code. For this, we're just going to alternate between entanglers and Euler rotations.

In [63]:
def build_circuit(self, var_params=None):
    """Construct the variational circuit to be used as our ansatz."""
    self.var_params = self.set_var_params(var_params)

    #Main process: build layers of variational Euler rotations + Entanglers
    circuit = EulerCircuit(self.n_qubits)
    for ii in range(self.n_layers):
        circuit += Entanglers(self.n_qubits)
        circuit += EulerCircuit(self.n_qubits)

    #Define the reference-state circuit
    reference_state_circuit = self.prepare_reference_state()
    
    #combine reference state with variational ansatz circuit
    if reference_state_circuit.size != 0:
        self.circuit = reference_state_circuit + circuit
    else:
        self.circuit = circuit
    return circuit

With the *Ansatz* so defined, we're ready to go ahead and build our VQE solver class, and run a calculation on a molecule of interest. I'm going to use pyscf to build a hydrogen dimer.

In [59]:
from pyscf import gto
H2 = [('H',(0,0,0)),('H',(0,0,0.74137727))]
mol_H2 = gto.Mole()
mol_H2.atom = H2
mol_H2.basis = "sto-3g"
mol_H2.charge = 0
mol_H2.spin = 0
mol_H2.build()

Great, we're ready to instantiate our ansatz, and feed it into VQE.

In [64]:
HEA_ansatz = HEA(n_qubits = 4, n_electrons = 2, n_layers = 4)
options = {"molecule": mol_H2, "qubit_mapping": 'jw', 'ansatz': HEA_ansatz}

HEA_VQE = VQESolver(options)
HEA_VQE.build()
HEA_VQE.simulate()

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -1.137257605548561
            Iterations: 38
            Function evaluations: 2358
            Gradient evaluations: 38


-1.137257605548561

Great, we were expecting -1.137270422018 Ha, and we got this to within 0.1 mHa.

So to summarize what we've done here, we defined an Ansatz class which had member methods allowing us to set and update variational parameters, and build both a reference-state and ansatz circuit.
This object was then plugged into the VQESolver to estimate the energy of our molecule of interest. 